# #305 Braced pair of girders
<i>Models a pair of braced steel girders and carries out linear buckling analysis</i>
***

![Preview](../_img/jupyter_notebook_305.png)

In [ ]:
'''Inputs - Units are N,mm'''
 # Each segment of the girder has a different section, defined below
segments_lengths = [6000, 5000, 6000, 3000, 3000, 6000, 5000, 6000]
# Index in the array of segment lengths indicating the central support (two span bridge)
index_centre_support = 4 
mesh_size = 250

transverse_spacing = 4000 # Spacing of the two main girders

super_imposed_load = 10.0 # Superimposed load (N/mm)

do_solve = False          # Carry out the analysis once the model is constructed
eigen_range = True        # Use a range for eigenvalue buckling analysis (avoids negative modes)


Create a girder class to hold the section dimensions for each segment</br>
We'll create points in the LUSAS model for each section and then use the points to create surfaces joining them.

In [ ]:
class GirderSection:
    def __init__(self, tfb:float, tfthk:float, dw:float, tw:float, bfb:float, bfthk:float):
        self.top_flange_breadth = tfb
        self.top_flange_thk = tfthk
        self.web_depth = dw
        self.web_thk = tw
        self.bottom_flange_breadth = bfb
        self.bottom_flange_thk = bfthk
        # We'll save the points created in the model in these lists so we can later use them to define surfaces
        self.bf_points = []
        self.tf_points = []


Create a list of section definitions, we'll need one per segment + 1 for the end.

In [ ]:
sections1 : list[GirderSection] = []
sections1.append(GirderSection(500, 30, 1500, 15, 600, 40))
sections1.append(GirderSection(500, 50, 1500, 10, 600, 60))
sections1.append(GirderSection(500, 30, 1500, 15, 600, 40))
sections1.append(GirderSection(500, 30, 1500, 20, 600, 50))
sections1.append(GirderSection(500, 30, 1800, 20, 600, 50)) # Central support
sections1.append(GirderSection(500, 30, 1500, 15, 600, 40))
sections1.append(GirderSection(500, 50, 1500, 10, 600, 60))
sections1.append(GirderSection(500, 30, 1500, 15, 600, 40))
sections1.append(GirderSection(500, 30, 1500, 15, 600, 40))
# Check we have defined one more section then segment lengths
assert(len(segments_lengths) == len(sections1)-1)
import copy
sections2 = [copy.deepcopy(s) for s in sections1]

sections_list = [sections1, sections2]

Create a list of stiffener thicknesses that will be positioned at each change in section. A zero thickness indicates no stiffener.

In [ ]:
stiffener_thicknesses = [20, 10, 10, 20, 25, 20, 10, 10, 20]

# Ensure number of stiffener thicknesses matches the number of segments
assert(len(sections1) == len(stiffener_thicknesses))

#### Connect to LUSAS Modeller and create a new model

In [ ]:
import sys; sys.path.append('../') # Reference modules in parent directory
from shared.LPI import *
lusas = get_lusas_modeller()
if lusas.existsDatabase() and lusas.db().isModified():
    raise Exception("This script will create a new model. Please save or close the current model and try again")

# Initialise the helpers module with a reference to LUSAS
import shared.Helpers as Helpers
Helpers.initialise(lusas)

# Create a new model
lusas.newProject("Structural", "Two_Span_Composite_Bridge.mdl")
# Get a reference to the current model database for convenience
db = lusas.database() 
# 3D model with Z vertical
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Units N,mm
db.setModelUnits("N,mm,t,s,C")

In [ ]:
# When calling the LPI from an external process as we are here a speedup can be gained in v22 and later by disabling the UI
if lusas.getMajorVersionNumber() >=22:
    lusas.enableUI(False)

Create model attributes

In [ ]:
# Create the mesh attribute
shell_mesh_attr = db.createMeshSurface(f"Shell Mesh {mesh_size}mm").setRegularSize("QTS4", mesh_size, True)

# Bar mesh attributes
bracing_mesh_attr = db.createMeshLine("Bar Mesh").setNumber("BRS2", 1)

# Steel Material attribute
steel_material = db.createIsotropicMaterial("Steel", 200_000, 0.3, 7.8e-9, 10e-6)

# Concrete Material attribute
concrete_material = db.createIsotropicMaterial("Concrete", 30_000, 0.2, 2.4e-9, 10e-6)

# Bracing geometic attribute
bracing_section_attr = db.createGeometricLine("Bracing Section")
bracing_section_attr.setFromLibrary("UK Sections", "Equal Angles (Advance)", "150x150x12 UKEA", 0, 0, 0)

# Support attributes
pinned = db.createSupportStructural("Fixed in XZ").setStructural("R", "F", "R", "F", "F", "F", "F", "F", "C", "F")
slide = db.createSupportStructural("Fixed in Z").setStructural("F", "F", "R", "F", "F", "F", "F", "F", "C", "F")

Create all the points for each section, saving the returned point in the Girder section definition. Use the helper functions for creating points

In [ ]:
s:GirderSection
for sections, y in [(sections1, -transverse_spacing/2),(sections2, transverse_spacing/2)]:
    x = 0
    for i in range(0, len(sections)):
        # Determine the longitudinal position (x coordinate) for each section
        if i > 0 : x += segments_lengths[i-1]
        # Determine the section to use at each longitudinal position
        s = sections[i]
        # Create the bottom flange points
        s.bf_points.append(Helpers.create_point(x, y + s.bottom_flange_breadth/2, -s.web_depth))
        s.bf_points.append(Helpers.create_point(x, y,                             -s.web_depth))
        s.bf_points.append(Helpers.create_point(x, y - s.bottom_flange_breadth/2, -s.web_depth))
        # Create the top flange points
        s.tf_points.append(Helpers.create_point(x, y +s.top_flange_breadth/2, 0.0))
        s.tf_points.append(Helpers.create_point(x, y,                         0.0))
        s.tf_points.append(Helpers.create_point(x, y -s.top_flange_breadth/2, 0.0))

Now join up the segment points. First create helper function to create surfaces and assign a geometric thickness attribute

In [ ]:
def create_surface(pnts: list, thk:float, ecc:float, group:str):

    geometry_data = lusas.geometryData().setAllDefaults()
    geometry_data.setLowerOrderGeometryType("points")

    obs = lusas.newObjectSet().add(pnts)

    surface = obs.createSurface(geometry_data).getObject("Surface")
    db.getGroupByName(group).add(surface, "Surfaces")

    name = f"{group} ({thk:.1f}mm)"

    db.createGeometricSurface(name).setSurface(thk, ecc).assignTo(surface)

Create groups to contain the girder surfaces

In [ ]:
db.createEmptyGroup("Top Flange")
db.createEmptyGroup("Web")
db.createEmptyGroup("Bottom Flange")
db.createEmptyGroup("Web Stiffeners")
db.createEmptyGroup("Deck")

Here we'll create the surfaces between each section definition using the points we created earlier

In [ ]:
for sections in sections_list:
    for i in range(0, len(segments_lengths)):

        s1:GirderSection = sections[i]   # Section definition at the start of the segment
        s2:GirderSection = sections[i+1] # Section definition at the end of the segment

        create_surface([ s1.bf_points[1], s2.bf_points[1], s2.bf_points[0], s1.bf_points[0] ], s1.bottom_flange_thk, 0.0, "Bottom Flange")
        create_surface([ s1.bf_points[2], s2.bf_points[2], s2.bf_points[0], s1.bf_points[0] ], s1.bottom_flange_thk, 0.0, "Bottom Flange")
        create_surface([ s1.bf_points[1], s2.bf_points[1], s2.tf_points[1], s1.tf_points[1] ], s1.web_thk,           0.0, "Web")
        create_surface([ s1.tf_points[1], s2.tf_points[1], s2.tf_points[0], s1.tf_points[0] ], s1.top_flange_thk,    0.0, "Top Flange")
        create_surface([ s1.tf_points[2], s2.tf_points[2], s2.tf_points[1], s1.tf_points[1] ], s1.top_flange_thk,    0.0, "Top Flange")


Create bearing stiffeners at the end sections

In [ ]:
for sections in sections_list:
    for i, thk in enumerate(stiffener_thicknesses):

        if thk > 0:

            s:GirderSection = sections[i]   # Section definition at the start of the segment
            create_surface([ s.bf_points[0], s.tf_points[0], s.tf_points[1], s.bf_points[1] ], thk, 0.0, "Web Stiffeners")
            create_surface([ s.bf_points[1], s.tf_points[1], s.tf_points[2], s.bf_points[2] ], thk, 0.0, "Web Stiffeners")


Assign a shell mesh attribute and steel material attribute to all surfaces

In [ ]:
# Assign mesh and steel material to all girder surfaces
assignment = lusas.assignment().setAllDefaults()
shell_mesh_attr.assignTo("Surfaces", assignment)
steel_material.assignTo("Surfaces", assignment)

Support attributes

(if you get a "Can't visualise attributes without a mesh" warning, just click OK. This can be avoided by assigning and updating the mesh first.)

In [ ]:
# To assign these support attributes we'll need to get hold of the lines. We can do this via the points that have been created for each section

# Loop through each member
for sections in sections_list:
    # Assign supports to the end and central section
    for i, section in enumerate([sections[0], sections[index_centre_support], sections[-1]]):

        p0 = section.bf_points[0] # point at the +ve flange edge
        p1 = section.bf_points[1] # point at the centre of the web.
        p2 = section.bf_points[2] # point at the -ve flange edge

        # Loop through connected lines 
        for line in p1.getHOFs():
            # Points of the connected line
            ps1 = line.getStartPoint()
            ps2 = line.getEndPoint()
            # Check if the connected line points are those defining the bottom flange
            # If so then we can assign the relevant support attribute
            if ps1 == p0 or ps1 == p2 or ps2 == p0 or ps2 == p2:
                pinned.assignTo(line) if i == 1 else slide.assignTo(line)


In [ ]:
# Set Default Attributes for bracing members
db.setAsDefault("Mesh", bracing_mesh_attr)
db.setAsDefault("Material", steel_material)
db.setAsDefault("Line Geometric", bracing_section_attr)

In [ ]:
#bracing_lines = []
for i in range(0, len(segments_lengths)+1):
    line1 = Helpers.create_line_from_points(sections1[i].bf_points[1], sections2[i].tf_points[1])
    line2 = Helpers.create_line_from_points(sections1[i].tf_points[1], sections2[i].bf_points[1])
    #lusas.newObjectSet().add(line1).add(line2).intersectLines(lusas.geometryData().setAllDefaults())

    line3 = Helpers.create_line_from_points(sections1[i].bf_points[1], sections2[i].bf_points[1])
    line4 = Helpers.create_line_from_points(sections1[i].tf_points[1], sections2[i].tf_points[1])

In [ ]:
# Clear default attribute assignments
db.setAsDefault("Mesh", "None")
db.setAsDefault("Material", "None")
db.setAsDefault("Line Geometric", "None")

# Loading

In [ ]:
# Get the automatically created loadcase in analysis 1 and add automatic gravity to it
# The loadcase will be used only to test the model is working correctly
gravity_loadcase : IFLoadcase = db.getLoadset("Loadcase 1", 0)
gravity_loadcase.setName("Gravity")
gravity_loadcase.addGravity(True)
db.getAnalysis("Analysis 1").setName("00 Base")

In [ ]:
# Create a new analysis for the linear buckling analysis
linear_analysis = db.createAnalysisStructural("01 Linear Buckling", False)

# Linear buckling loadcase
linear_buckling_loadcase = db.createLoadcase("Linear Eigen Buckling", linear_analysis.getName())
linear_buckling_loadcase.addGravity(True)

# Set the loadcase controls to be a Buckling analysis searching for the minimum 5 buckling modes
linear_buckling_loadcase.setEigenvalueMaxMinControl("Buckling", "Minimum", 5)

In [ ]:
# Helper function to find the lines in the top flanges to which superimposed loads will be assigned
def get_lines_x(points:list[IFPoint])->list[IFLine]:
    lines = []
    for point in points:
        for hof in point.getHOFs():
            if hof.getTypeCode() == 2 : # Line
                # Check it is aligned with and parallel to the x axis
                if abs(hof.getEndPosition()[1] - hof.getStartPosition()[1]) < 1e-3 and \
                   abs(hof.getEndPosition()[2] - hof.getStartPosition()[2]) < 1e-3:
                    # Add to the list if not already in there
                    if not hof in lines:
                        lines.append(hof)
    return lines

# List of lines in section 1 top flange
lines_s1_tf   = get_lines_x([s.tf_points[1] for s in sections1])
# List of lines in section 1 top flange
lines_s2_tf   = get_lines_x([s.tf_points[1] for s in sections2])

In [ ]:
# Create a new loadcase for the global distributed load
load_attr = db.createLoadingGlobalDistributed("Global Distributed Load")
load_attr.setGlobalDistributed("Length", 0.0, 0.0, -super_imposed_load, 0.0, 0.0, 0.0, 0.0, 0.0, False, 0.0)

# Assign the load to the top flange surfaces, in both linear and nonlinear analyses
load_attr.assignTo(lines_s1_tf, lusas.assignment().setAllDefaults().setLoadset(linear_buckling_loadcase))
load_attr.assignTo(lines_s2_tf, lusas.assignment().setAllDefaults().setLoadset(linear_buckling_loadcase))

Set the model to display the assigned plate thicknesses and supports in an isometric orientation

In [ ]:
lusas.view().insertGeometryLayer() # Make sure the drawing layers exist before accessing them
lusas.view().insertAttributesLayer()

lusas.view().geometry().autoColourByAttributes("Geometric", True)
lusas.view().attributes().visualiseAll("Supports")
lusas.view().setIsometric()

In [ ]:
# Mesh the model
db.resetMesh()
db.updateMesh()

In [ ]:
# Solve all the analyses
if do_solve:
    for analysis in db.getAnalyses():
        analysis.solve(True)
    db.openAllResults(False)

In [ ]:
# Re-enable the UI. Note if the script failed prior to this, this cell must be run manually to re-enable the UI
lusas.enableUI(True)